In [1]:
import pandas as pd
import numpy as np
import random
import math
import copy

In [3]:
data = pd.read_excel("kepsekfixgarandom.xls")
data.head()

,Nomor Urut,Kode_Sekolah,Nama Sekolah,Nomor_guru,Nama Guru,Jarak rmh-sek
0,1,S1,SD PL1,G0001,John Fonataba,63
1,2,S1,SD PL1,G0002,Rianny Linthin,95
2,3,S1,SD PL1,G0003,Richard Tumewu,53
3,4,S1,SD PL1,G0004,Paijo,60
4,5,S1,SD PL1,G0005,Sriningsih,57


# Lanjutan

In [4]:
s = data["Kode_Sekolah"].drop_duplicates()
s.reset_index(drop=True, inplace = True)

In [5]:
data_konfig = data.iloc[0:0,:].copy()
isi = data.iloc[0:0,:].copy()
guru ='' 
for i in s:
  if(data_konfig.empty):
    isi = data[data["Kode_Sekolah"]==i].sample()
    guru = str(isi["Nama Guru"].any())
    data_konfig = data_konfig.append(isi)
  else:
    while(guru in str(data_konfig["Nama Guru"].values)):
      isi = data[data["Kode_Sekolah"]==i].sample()
      guru = str(isi["Nama Guru"].any())
    data_konfig = data_konfig.append(isi)
data_konfig

,Nomor Urut,Kode_Sekolah,Nama Sekolah,Nomor_guru,Nama Guru,Jarak rmh-sek
5,6,S1,SD PL1,G0006,Taufan,35
8,9,S2,SD KNS 1,G0003,Richard Tumewu,56
13,14,S3,SD PL2,G0002,Rianny Linthin,75
21,22,S4,SD KNS2,G0004,Paijo,94
24,25,S5,SD PL3,G0001,John Fonataba,44
34,35,S6,SD KLSN1,G0005,Sriningsih,91


In [6]:
def new_konf_rand_check(konfig, placement):
  w = []
  w_copy = []
  kode = []
  idx_source = []
  
  cek = konfig["Nama Guru"].to_numpy()
  r_pick = random.sample(set(cek), 2)
  r_pick
  
  new_konf=konfig.copy()
  new_konf.reset_index(drop=True, inplace = True)
  
  for i in range(2):
    nilai= np.where(cek == r_pick[i])
    nilai = nilai[0][0]
    w.append(nilai)
  
  w_copy = w.copy()
  temp = w[0]
  w[0] = w[1]
  w[1] = temp
  
  for i in range(2):
    if(w[i] == 0):
      kode.append('S1')
    elif(w[i]==1):
      kode.append('S2')
    elif(w[i]==2):
      kode.append('S3')
    elif(w[i]==3):
      kode.append('S4')
    elif(w[i]==4):
      kode.append('S5')
    elif(w[i]==5):
      kode.append('S6')
    
    condition = (data["Kode_Sekolah"]==kode[i]) & (data["Nama Guru"]==r_pick[i])
    idx_source.append(data.index[condition].tolist()[0])
    new_konf.iloc[w_copy[i]] = data.loc[idx_source[i]]
  
  new_konf = new_konf.sort_values(by=["Kode_Sekolah"],ascending = True)
  new_konf.reset_index(drop=True, inplace = True)
  #new_konf_name = new_konf["Nama Guru"].values.tolist()
  if(new_konf["Nama Guru"].values.tolist() in placement):
    new_konf_rand_check(konfig,placement)
  elif(new_konf["Nama Guru"].values.tolist() not in placement):
    #new_konf_namefix = new_konf_name
    new_konf = pd.DataFrame(new_konf, columns=['Nomor Urut', 'Kode_Sekolah','Nama Sekolah',
                                               'Nomor_guru', 'Nama Guru', 'Jarak rmh-sek'])
  return new_konf

In [9]:
def simulated_annealing(iteration, a):
  konfig = data_konfig
  placement = []
  total_distance = []
  t = []
  cek_placement_awal = []
  new_konfig = konfig.iloc[0:0,:].copy()
  
  placement.append(konfig["Nama Guru"].tolist())
  total_distance.append(konfig["Jarak rmh-sek"].sum())
  t.append(a*total_distance[0])

  for i in range(iteration):
    new_konfig = new_konf_rand_check(konfig, placement)
    #atarashi.append(new_konfig)
    total_distance.append(new_konfig["Jarak rmh-sek"].sum())

    if(total_distance[i+1] < total_distance[i]):
      placement.append(new_konfig["Nama Guru"].tolist())    
      t.append(a*total_distance[i])
    elif(total_distance[i+1] >= total_distance[i]):
      r = random.randint(0,100)/100
      p = math.exp(total_distance[i]-total_distance[i+1]/t[i])
      if(r<p):
        placement.append(new_konfig["Nama Guru"].tolist())
        t.append(a*total_distance[i])
      else:
        placement.append(placement[i])
        t.append(t[i])
        total_distance[-1] = total_distance[i]

  print(t)
  print(total_distance)
  print(placement)
  pl = copy.deepcopy(placement)
  singkatan = []
  for i in range(11):
    for j in range(6):
      if(pl[i][j]=='Sriningsih'):
        pl[i][j]='S'
      elif(pl[i][j]=='Rianny Linthin'):
        pl[i][j]='RL'
      elif(pl[i][j]=='Paijo'):
        pl[i][j]='P'
      elif(pl[i][j]=='Richard Tumewu'):
        pl[i][j]='RT'
      elif(pl[i][j]=='John Fonataba'):
        pl[i][j]='JF'
      elif(pl[i][j]=='Taufan'):
        pl[i][j]='T'
    singkatan.append(pl[i][0]+" ㅡ "+
                pl[i][1]+" ㅡ "+
                pl[i][2]+" ㅡ "+
                pl[i][3]+" ㅡ "+
                pl[i][4]+" ㅡ "+
                pl[i][5])
  
  bahan = {'Placement': singkatan, 'Total Distance': total_distance,'Temperature': t}
  SA = pd.DataFrame(bahan, columns= ['Placement', 'Total Distance', 'Temperature'])  
  return SA, cek_placement_awal, placement

In [10]:
hasil, awalmula, pnmptn = simulated_annealing(10,0.95)
hasil

[375.25, 375.25, 409.45, 409.45, 323.95, 293.55, 379.04999999999995, 378.09999999999997, 303.05, 394.25, 352.45]
[395, 431, 431, 341, 309, 399, 398, 319, 415, 371, 299]
[['Taufan', 'Richard Tumewu', 'Rianny Linthin', 'Paijo', 'John Fonataba', 'Sriningsih'], ['John Fonataba', 'Richard Tumewu', 'Rianny Linthin', 'Paijo', 'Taufan', 'Sriningsih'], ['John Fonataba', 'Richard Tumewu', 'Rianny Linthin', 'Paijo', 'Taufan', 'Sriningsih'], ['Taufan', 'Richard Tumewu', 'Sriningsih', 'Paijo', 'John Fonataba', 'Rianny Linthin'], ['Taufan', 'Richard Tumewu', 'Rianny Linthin', 'Sriningsih', 'John Fonataba', 'Paijo'], ['Taufan', 'Richard Tumewu', 'Rianny Linthin', 'John Fonataba', 'Paijo', 'Sriningsih'], ['Richard Tumewu', 'Taufan', 'Rianny Linthin', 'Paijo', 'John Fonataba', 'Sriningsih'], ['Taufan', 'Richard Tumewu', 'Paijo', 'Rianny Linthin', 'John Fonataba', 'Sriningsih'], ['Taufan', 'Richard Tumewu', 'John Fonataba', 'Paijo', 'Rianny Linthin', 'Sriningsih'], ['Paijo', 'Richard Tumewu', 'Rianny Li

,Placement,Total Distance,Temperature
0,T ㅡ RT ㅡ RL ㅡ P ㅡ JF ㅡ S,395,375.25
1,JF ㅡ RT ㅡ RL ㅡ P ㅡ T ㅡ S,431,375.25
2,JF ㅡ RT ㅡ RL ㅡ P ㅡ T ㅡ S,431,409.45
3,T ㅡ RT ㅡ S ㅡ P ㅡ JF ㅡ RL,341,409.45
4,T ㅡ RT ㅡ RL ㅡ S ㅡ JF ㅡ P,309,323.95
5,T ㅡ RT ㅡ RL ㅡ JF ㅡ P ㅡ S,399,293.55
6,RT ㅡ T ㅡ RL ㅡ P ㅡ JF ㅡ S,398,379.05
7,T ㅡ RT ㅡ P ㅡ RL ㅡ JF ㅡ S,319,378.10
8,T ㅡ RT ㅡ JF ㅡ P ㅡ RL ㅡ S,415,303.05
9,P ㅡ RT ㅡ RL ㅡ T ㅡ JF ㅡ S,371,394.25


In [ ]:
hasil.sort_values(by=["Total Distance"],ascending = True)

,Placement,Total Distance,Temperature
7,S ㅡ T ㅡ RT ㅡ JF ㅡ RL ㅡ P,238,330.60
3,P ㅡ T ㅡ RT ㅡ JF ㅡ S ㅡ RL,273,374.30
10,S ㅡ T ㅡ RL ㅡ JF ㅡ P ㅡ RT,345,380.95
6,RT ㅡ T ㅡ S ㅡ JF ㅡ P ㅡ RL,348,339.15
0,S ㅡ T ㅡ RT ㅡ JF ㅡ P ㅡ RL,349,331.55
5,T ㅡ S ㅡ RT ㅡ JF ㅡ P ㅡ RL,357,380.95
1,S ㅡ JF ㅡ RT ㅡ T ㅡ P ㅡ RL,358,331.55
2,JF ㅡ T ㅡ RT ㅡ S ㅡ P ㅡ RL,394,340.10
4,RL ㅡ T ㅡ RT ㅡ JF ㅡ P ㅡ S,401,259.35
8,S ㅡ RT ㅡ T ㅡ JF ㅡ P ㅡ RL,401,226.10


pada iterasi 4 dan 5 konfigurasi sama karena total distance konfig 5 sblmnya tidak memenuhi syarat ketika dihitung nilai probnya (harus memenuhi r < p), sehingga konfig5 menggunakan konfig iterasi sblmnya yaitu konfig4

In [ ]:
hasil[hasil["Total Distance"]==hasil["Total Distance"].min()]

,Placement,Total Distance,Temperature
7,S ㅡ T ㅡ RT ㅡ JF ㅡ RL ㅡ P,238,330.6


# coba2 aja

In [ ]:
s1 = data["Kode_Sekolah"].str.contains('S1', case=True, regex=True)
s2 = data["Kode_Sekolah"].str.contains('S2', case=True, regex=True)
s3 = data["Kode_Sekolah"].str.contains('S3', case=True, regex=True)
s4 = data["Kode_Sekolah"].str.contains('S4', case=True, regex=True)
s5 = data["Kode_Sekolah"].str.contains('S5', case=True, regex=True)
s6 = data["Kode_Sekolah"].str.contains('S6', case=True, regex=True)

In [ ]:
ks = data["Kode_Sekolah"].drop_duplicates()
ks.index
ks

0     S1
6     S2
12    S3
18    S4
24    S5
30    S6
Name: Kode_Sekolah, dtype: object

In [ ]:
data[s1]

,Nomor Urut,Kode_Sekolah,Nama Sekolah,Nomor_guru,Nama Guru,Jarak rmh-sek
0,1,S1,SD PL1,G0001,John Fonataba,63
1,2,S1,SD PL1,G0002,Rianny Linthin,95
2,3,S1,SD PL1,G0003,Richard Tumewu,53
3,4,S1,SD PL1,G0004,Paijo,60
4,5,S1,SD PL1,G0005,Sriningsih,57
5,6,S1,SD PL1,G0006,Taufan,35


In [ ]:
data_konfig = data.iloc[0:0,:].copy()
data_konfig = data_konfig.append(data[data["Kode_Sekolah"]==s[0]].sample())
data_konfig = data_konfig.append(data[data["Kode_Sekolah"]==s[1]].sample())
data_konfig

,Nomor Urut,Kode_Sekolah,Nama Sekolah,Nomor_guru,Nama Guru,Jarak rmh-sek
0,1,S1,SD PL1,G0001,John Fonataba,63
10,11,S2,SD KNS 1,G0005,Sriningsih,71


In [ ]:

# import pandas library
import pandas as pd
  
# dictionary with list object in values
details = {
    'Name' : ['Ankit', 'Aishwarya', 'Shaurya', 
              'Shivangi', 'Priya', 'Swapnil'],
    'Age' : [23, 21, 22, 21, 24, 25],
    'University' : ['BHU', 'JNU', 'DU', 'BHU', 'Geu', 'Geu'],
}
  
# creating a Dataframe object 
df = pd.DataFrame(details, columns = ['Name', 'Age', 'University'],
                  index = ['a', 'b', 'c', 'd', 'e', 'f'])
  
print("Dataframe: \n\n", df)
  
# chcek 'Ankit' exist in dataframe or not
if 'Ankit' in df.values :
    print("\nThis value exists in Dataframe")
  
else :
    print("\nThis value does not exists in Dataframe")

Dataframe: 

         Name  Age University
a      Ankit   23        BHU
b  Aishwarya   21        JNU
c    Shaurya   22         DU
d   Shivangi   21        BHU
e      Priya   24        Geu
f    Swapnil   25        Geu

This value exists in Dataframe


In [ ]:
empoyees = [('jack', 34, 'Sydney', 155) ,
            ('Riti', 31, 'Delhi' , 177) ,
            ('Aadi', 16, 'Mumbai', 81) ,
            ('Mohit', 31,'Delhi' , 167) ,
            ('Veena', 81, 'Delhi' , 144) ,
            ('Shaunak', 35, 'Mumbai', 135 ),
            ('Shaun', 35, 'Colombo', 111),
            ('Riti', 32, 'Colombo', 111),
            ]
# Create a DataFrame object
empDfObj = pd.DataFrame(empoyees, columns=['Name', 'Age', 'City', 'Marks'])
print('Aadi' in empDfObj["Name"].unique())


True


In [ ]:
import random
n = random.randint(0,100)/100
print(n)

0.53


In [ ]:
coba1 = ['yoi', 'haha', 'masasi', 'lah']
coba2 = ['haha', 'yoi', 'masasi', 'lah']
bigcoba = [coba1, coba2]
coba3 = ['haha', 'yoi', 'masasi', 'lah']
for i in bigcoba:
  if (coba3 in bigcoba):
    print("identical")
  else:
    print("not identical")

identical
identical


In [ ]:
w = []
kode = []
idx_source = []
for i in range(2):
  nilai= np.where(cek == r_pick[i])
  nilai = nilai[0][0]
  w.append(nilai)
temp = w[0]
w[0] = w[1]
w[1] = temp
for i in range(2):
  if(w[i] == 0):
    kode.append('S1')
  elif(w[i]==1):
    kode.append('S2')
  elif(w[i]==2):
    kode.append('S3')
  elif(w[i]==3):
    kode.append('S4')
  elif(w[i]==4):
    kode.append('S5')
  elif(w[i]==5):
    kode.append('S6')
kode

['S2', 'S5']

In [ ]:
for i in range(2):
  condition = (data["Kode_Sekolah"]==kode[i]) & (data["Nama Guru"]==r_pick[i])
  #print(data.index[condition].tolist())
  idx_source.append(data.index[condition].tolist()[0])
idx_source

In [ ]:
cek4 = ['Sriningsih',
 'Taufan',
 'Rianny Linthin',
 'Paijo',
 'Richard Tumewu',
 'John Fonataba']
cek4

['Sriningsih',
 'Taufan',
 'Rianny Linthin',
 'Paijo',
 'Richard Tumewu',
 'John Fonataba']

In [ ]:
cek2 = data_konfig["Nama Guru"].values.tolist()
cek2

['Sriningsih',
 'Richard Tumewu',
 'Rianny Linthin',
 'Paijo',
 'Taufan',
 'John Fonataba']

In [ ]:
bigcek = [cek2, cek4]
if(cek3 in bigcek):
  print("ha ada")
else:
  print("gaada di bigcek")

ha ada


In [ ]:
cek3 = new_konf["Nama Guru"].values.tolist()
cek3

['Sriningsih',
 'Taufan',
 'Rianny Linthin',
 'Paijo',
 'Richard Tumewu',
 'John Fonataba']

In [ ]:
products = {'Product': ['Tablet','iPhone','Laptop','Monitor'],
            'Price': [250,800,1200,300]
            }

df = pd.DataFrame(products, columns= ['Product', 'Price'])
df

,Product,Price
0,Tablet,250
1,iPhone,800
2,Laptop,1200
3,Monitor,300


In [ ]:
dt = {'names': ['r','b'], 'formats': ['u1', 'u1'],
               'offsets': [0, 2],
               'titles': ['Red pixel', 'Blue pixel']}
dn = pd.DataFrame(dt, columns= ['names', 'formats', 'offsets','titles'])
dn

,names,formats,offsets,titles
0,r,u1,0,Red pixel
1,b,u1,2,Blue pixel


In [ ]:
hehe = []
for i in range(11):
  for j in range(6):
    #print(i, end =' ')
   # print(j)
    if(pl[i][j]=='Sriningsih'):
      pl[i][j]='S'
    elif(pl[i][j]=='Rianny Linthin'):
      pl[i][j]='RL'
    elif(pl[i][j]=='Paijo'):
      pl[i][j]='P'
    elif(pl[i][j]=='Richard Tumewu'):
      pl[i][j]='RT'
    elif(pl[i][j]=='John Fonataba'):
      pl[i][j]='JF'
    elif(pl[i][j]=='Taufan'):
      pl[i][j]='T'
  hehe.append(pl[i][0]+" ㅡ "+pl[i][1]+" ㅡ "+pl[i][2]+" ㅡ "+pl[i][3]+" ㅡ "+pl[i][4]+" ㅡ "+pl[i][5])
hehe

['S ㅡ RL ㅡ P ㅡ JF ㅡ RT ㅡ T',
 'S ㅡ RL ㅡ RT ㅡ JF ㅡ P ㅡ T',
 'S ㅡ RL ㅡ P ㅡ JF ㅡ T ㅡ RT',
 'S ㅡ RL ㅡ RT ㅡ JF ㅡ P ㅡ T',
 'S ㅡ T ㅡ P ㅡ JF ㅡ RT ㅡ RL',
 'S ㅡ JF ㅡ P ㅡ RL ㅡ RT ㅡ T',
 'S ㅡ RL ㅡ JF ㅡ P ㅡ RT ㅡ T',
 'S ㅡ RL ㅡ JF ㅡ P ㅡ RT ㅡ T',
 'S ㅡ RL ㅡ JF ㅡ P ㅡ RT ㅡ T',
 'JF ㅡ RL ㅡ P ㅡ S ㅡ RT ㅡ T',
 'S ㅡ RL ㅡ P ㅡ T ㅡ RT ㅡ JF']

In [ ]:
w = []
w_copy = []
kode = []
idx_source = []

cek = data_konfig["Nama Guru"].to_numpy()
r_pick = random.sample(set(cek), 2)
r_pick

new_konf=data_konfig.copy()
new_konf.reset_index(drop=True, inplace = True)

for i in range(2):
  nilai= np.where(cek == r_pick[i])
  nilai = nilai[0][0]
  w.append(nilai)

w_copy = w.copy()
temp = w[0]
w[0] = w[1]
w[1] = temp

for i in range(2):
  if(w[i] == 0):
    kode.append('S1')
  elif(w[i]==1):
    kode.append('S2')
  elif(w[i]==2):
    kode.append('S3')
  elif(w[i]==3):
    kode.append('S4')
  elif(w[i]==4):
    kode.append('S5')
  elif(w[i]==5):
    kode.append('S6')
    
  condition = (data["Kode_Sekolah"]==kode[i]) & (data["Nama Guru"]==r_pick[i])
  idx_source.append(data.index[condition].tolist()[0])
  new_konf.iloc[w_copy[i]] = data.loc[idx_source[i]]
  
new_konf = new_konf.sort_values(by=["Kode_Sekolah"],ascending = True)
new_konf.reset_index(drop=True, inplace = True)
  #new_konf_name = new_konf["Nama Guru"].values.tolist()
if(new_konf["Nama Guru"].values.tolist() in placement):
  new_konf_rand_check(data_konfig,placement)
elif(new_konf["Nama Guru"].values.tolist() not in placement):
    #new_konf_namefix = new_konf_name
  new_konf = pd.DataFrame(new_konf, columns=['Nomor Urut', 'Kode_Sekolah','Nama Sekolah','Nomor_guru', 'Nama Guru', 'Jarak rmh-sek'])
new_konf

,Nomor Urut,Kode_Sekolah,Nama Sekolah,Nomor_guru,Nama Guru,Jarak rmh-sek
0,5,S1,SD PL1,G0005,Sriningsih,57
1,7,S2,SD KNS 1,G0001,John Fonataba,51
2,15,S3,SD PL2,G0003,Richard Tumewu,32
3,24,S4,SD KNS2,G0006,Taufan,45
4,26,S5,SD PL3,G0002,Rianny Linthin,48
5,34,S6,SD KLSN1,G0004,Paijo,14


# jaga2

In [ ]:
awalmula

[   Nomor Urut Kode_Sekolah  ...       Nama Guru Jarak rmh-sek
 0           2           S1  ...  Rianny Linthin            95
 1          11           S2  ...      Sriningsih            71
 2          15           S3  ...  Richard Tumewu            32
 3          24           S4  ...          Taufan            45
 4          25           S5  ...   John Fonataba            44
 5          34           S6  ...           Paijo            14
 
 [6 rows x 6 columns],
    Nomor Urut Kode_Sekolah  ...       Nama Guru Jarak rmh-sek
 0           5           S1  ...      Sriningsih            57
 1          10           S2  ...           Paijo            99
 2          15           S3  ...  Richard Tumewu            32
 3          24           S4  ...          Taufan            45
 4          25           S5  ...   John Fonataba            44
 5          32           S6  ...  Rianny Linthin            77
 
 [6 rows x 6 columns],
    Nomor Urut Kode_Sekolah  ...       Nama Guru Jarak rmh-sek
 0   

In [ ]:
SIMA, pl= simulated_annealing(10,0.95)

[276.45, 338.2, 232.75, 338.2, 269.8, 312.55, 393.29999999999995, 393.29999999999995, 393.29999999999995, 319.2, 219.45]
[291, 356, 245, 356, 284, 329, 414, 414, 414, 336, 231]
[['Sriningsih', 'Rianny Linthin', 'Paijo', 'John Fonataba', 'Richard Tumewu', 'Taufan'], ['Sriningsih', 'Rianny Linthin', 'Richard Tumewu', 'John Fonataba', 'Paijo', 'Taufan'], ['Sriningsih', 'Rianny Linthin', 'Paijo', 'John Fonataba', 'Taufan', 'Richard Tumewu'], ['Sriningsih', 'Rianny Linthin', 'Richard Tumewu', 'John Fonataba', 'Paijo', 'Taufan'], ['Sriningsih', 'Taufan', 'Paijo', 'John Fonataba', 'Richard Tumewu', 'Rianny Linthin'], ['Sriningsih', 'John Fonataba', 'Paijo', 'Rianny Linthin', 'Richard Tumewu', 'Taufan'], ['Sriningsih', 'Rianny Linthin', 'John Fonataba', 'Paijo', 'Richard Tumewu', 'Taufan'], ['Sriningsih', 'Rianny Linthin', 'John Fonataba', 'Paijo', 'Richard Tumewu', 'Taufan'], ['Sriningsih', 'Rianny Linthin', 'John Fonataba', 'Paijo', 'Richard Tumewu', 'Taufan'], ['John Fonataba', 'Rianny Lint

In [ ]:
placement = []
for i in range(11):
  placement.append(data_konfig["Nama Guru"].tolist())
  yoohoo = new_konf_rand_check(data_konfig, placement)
  print(yoohoo)

[[3, 'S1', 'SD PL1', 'G0003', 'Richard Tumewu', 53], [8, 'S2', 'SD KNS 1', 'G0002', 'Rianny Linthin', 44], [13, 'S3', 'SD PL2', 'G0001', 'John Fonataba', 91], [22, 'S4', 'SD KNS2', 'G0004', 'Paijo', 94], [30, 'S5', 'SD PL3', 'G0006', 'Taufan', 52], [35, 'S6', 'SD KLSN1', 'G0005', 'Sriningsih', 91]]
[[3, 'S1', 'SD PL1', 'G0003', 'Richard Tumewu', 53], [11, 'S2', 'SD KNS 1', 'G0005', 'Sriningsih', 71], [13, 'S3', 'SD PL2', 'G0001', 'John Fonataba', 91], [22, 'S4', 'SD KNS2', 'G0004', 'Paijo', 94], [26, 'S5', 'SD PL3', 'G0002', 'Rianny Linthin', 48], [36, 'S6', 'SD KLSN1', 'G0006', 'Taufan', 81]]
[[3, 'S1', 'SD PL1', 'G0003', 'Richard Tumewu', 53], [12, 'S2', 'SD KNS 1', 'G0006', 'Taufan', 41], [13, 'S3', 'SD PL2', 'G0001', 'John Fonataba', 91], [23, 'S4', 'SD KNS2', 'G0005', 'Sriningsih', 85], [26, 'S5', 'SD PL3', 'G0002', 'Rianny Linthin', 48], [34, 'S6', 'SD KLSN1', 'G0004', 'Paijo', 14]]
[[3, 'S1', 'SD PL1', 'G0003', 'Richard Tumewu', 53], [12, 'S2', 'SD KNS 1', 'G0006', 'Taufan', 41]

In [ ]:
data_konfig["Nama Guru"].values.tolist()

['Paijo',
 'Richard Tumewu',
 'Taufan',
 'John Fonataba',
 'Rianny Linthin',
 'Sriningsih']

In [ ]:
data.to_excel("kepsekfixgarandom.xls", index = False)

In [ ]:
pnmptn

[['Sriningsih',
  'Taufan',
  'Richard Tumewu',
  'John Fonataba',
  'Paijo',
  'Rianny Linthin'],
 ['Sriningsih',
  'John Fonataba',
  'Richard Tumewu',
  'Taufan',
  'Paijo',
  'Rianny Linthin'],
 ['John Fonataba',
  'Taufan',
  'Richard Tumewu',
  'Sriningsih',
  'Paijo',
  'Rianny Linthin'],
 ['Paijo',
  'Taufan',
  'Richard Tumewu',
  'John Fonataba',
  'Sriningsih',
  'Rianny Linthin'],
 ['Rianny Linthin',
  'Taufan',
  'Richard Tumewu',
  'John Fonataba',
  'Paijo',
  'Sriningsih'],
 ['Taufan',
  'Sriningsih',
  'Richard Tumewu',
  'John Fonataba',
  'Paijo',
  'Rianny Linthin'],
 ['Richard Tumewu',
  'Taufan',
  'Sriningsih',
  'John Fonataba',
  'Paijo',
  'Rianny Linthin'],
 ['Sriningsih',
  'Taufan',
  'Richard Tumewu',
  'John Fonataba',
  'Rianny Linthin',
  'Paijo'],
 ['Sriningsih',
  'Richard Tumewu',
  'Taufan',
  'John Fonataba',
  'Paijo',
  'Rianny Linthin'],
 ['Rianny Linthin',
  'Taufan',
  'Richard Tumewu',
  'John Fonataba',
  'Paijo',
  'Sriningsih'],
 ['Srining